In [ ]:
import cv2
import torch
import numpy as np
from deepface import DeepFace
from ultralytics import YOLO


yolo_model = YOLO("yolov8n-face.pt")  

def detect_faces(image):
    """Detect faces using YOLOv8 and return cropped face images."""
    results = yolo_model(image)
    faces = []
    
    for result in results:
        boxes = result.boxes.xyxy.cpu().numpy()  # Get bounding boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box)
            face = image[y1:y2, x1:x2]
            faces.append(face)
    
    return faces

def extract_face_embedding(face):
    """Extract 128D face embeddings using DeepFace (Facenet)."""
    embedding = DeepFace.represent(face, model_name="Facenet", enforce_detection=False)
    return embedding[0]['embedding'] if embedding else None

def compare_faces(reference_embedding, test_embedding, threshold=0.6):
    """Compare two face embeddings using cosine similarity."""
    if reference_embedding is None or test_embedding is None:
        return False  # Cannot compare if embedding extraction failed
    similarity = np.dot(reference_embedding, test_embedding) / (np.linalg.norm(reference_embedding) * np.linalg.norm(test_embedding))
    return similarity > threshold  # True if similarity is above threshold

def process_video(video_path, reference_embedding):
    """Process video and check if the person is detected in any frame."""
    cap = cv2.VideoCapture(video_path)
    detected = False
    frame_count = 0
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_count += 1
        if frame_count % 10 != 0:  # Process every 10th frame for efficiency
            continue
        
        faces = detect_faces(frame)
        for face in faces:
            embedding = extract_face_embedding(face)
            if compare_faces(reference_embedding, embedding):
                detected = True
                break  # Stop checking further frames if found
        
        if detected:
            break  # Stop processing video if detected
    
    cap.release()
    return detected


In [2]:

# Upload an image of the person (Reference Image)
reference_image = cv2.imread("1.jpg")  # Provide reference image path
reference_faces = detect_faces(reference_image)
reference_embedding = extract_face_embedding(reference_faces[0]) if reference_faces else None

# Upload a video (CCTV footage)
video_path = "test_.mp4"  # Provide video file path
if reference_embedding:
    is_present = process_video(video_path, reference_embedding)
    print("Student Detected in CCTV Footage!" if is_present else "Student Not Found!")
else:
    print("Error: No face detected in the reference image.")


FileNotFoundError: [Errno 2] No such file or directory: '1.jpg'